In [ ]:
!pip -q install transformers torch accelerate
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 55.1 MB/s eta 0:00:00


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoConfig
import pandas as pd
import random

In [ ]:
!gdown 152eRXERoaQE02K6yAnA7EvouVRMEp-bH

Downloading...
From: https://drive.google.com/uc?id=152eRXERoaQE02K6yAnA7EvouVRMEp-bH
To: /content/train_processed.csv
100% 36.1M/36.1M [00:00<00:00, 101MB/s]


In [ ]:
df = pd.read_csv("/content/train_processed.csv")
df.head(3)

,score,word_count,essay_length,sentences_count,paragraph_count,clean_text,flesch_reading_ease,flesch_kincaid_grade,smog_index,coleman_liau_index,automated_readability_index,dale_chall_readability_score,difficult_words,linsear_write_formula,misspelled_count
0,3,498,2677,13,1,many people have car where they live. the thin...,56.97,15.1,11.4,7.21,18.0,8.40,57.0,13.000000,26
1,3,332,1669,19,9,i am a scientist at nasa that is discussing th...,87.35,5.5,6.8,4.52,5.9,6.11,22.0,6.714286,8
2,4,550,3077,24,7,people always wish they had the same technolog...,65.05,9.9,11.4,8.36,11.1,6.92,59.0,15.500000,9


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17307 entries, 0 to 17306
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   score                         17307 non-null  int64  
 1   word_count                    17307 non-null  int64  
 2   essay_length                  17307 non-null  int64  
 3   sentences_count               17307 non-null  int64  
 4   paragraph_count               17307 non-null  int64  
 5   clean_text                    17307 non-null  object 
 6   flesch_reading_ease           17307 non-null  float64
 7   flesch_kincaid_grade          17307 non-null  float64
 8   smog_index                    17307 non-null  float64
 9   coleman_liau_index            17307 non-null  float64
 10  automated_readability_index   17307 non-null  float64
 11  dale_chall_readability_score  17307 non-null  float64
 12  difficult_words               17307 non-null  float64
 13  l

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

# cấu hình cho model
config = AutoConfig.from_pretrained("google/flan-t5-base", trust_remote_code = True)
config.init_device = "cuda"
config.temperature = 0.1
config.max_length = 300
config.eos_token_id = tokenizer.eos_token_id
config.pad_token_id = tokenizer.pad_token_id
config.do_sample = True

model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", device_map = "cuda", config = config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Choose a random 'clean_text' row from the dataframe as the input text
input_text = df['clean_text'][random.randint(0, len(df))]
print("Input text:", input_text)

Input text: driverless car are something that will very soon be a common part of our everyday furture life. these car have proven to be a time saving and fuel saving option. they also allow driver to not have to be in full control a they would have had in normal car today. although these would be some very helpful thing and would be more economically friendly, driverless car prove to be very risking and there are just too many question about ensuring safety for them to be allowed today. driverless car could possibly be dangerous to the driver, other driver, or people walking around on the street. a of now no driverless car have been developed that are completely driverless. at any point the car could run into a problem where it need a person is assistence. this person may not always pay attention like they should and they may not be able to interfere with the problem quick enought to prevent harm. in these car the chance of something to go wrong is likely. an accident may occur and the

In [ ]:
summary_command = 'Summary this text: {}'.format(input_text)
print('-> Command:', summary_command)

input_ids = tokenizer(summary_command, return_tensors = "pt").input_ids.to("cuda")

outputs = model.generate(input_ids)
print('\n->Answer:\n', tokenizer.decode(outputs[0]))

Token indices sequence length is longer than the specified maximum sequence length for this model (602 > 512). Running this sequence through the model will result in indexing errors


-> Command: Summary this text: driverless car are something that will very soon be a common part of our everyday furture life. these car have proven to be a time saving and fuel saving option. they also allow driver to not have to be in full control a they would have had in normal car today. although these would be some very helpful thing and would be more economically friendly, driverless car prove to be very risking and there are just too many question about ensuring safety for them to be allowed today. driverless car could possibly be dangerous to the driver, other driver, or people walking around on the street. a of now no driverless car have been developed that are completely driverless. at any point the car could run into a problem where it need a person is assistence. this person may not always pay attention like they should and they may not be able to interfere with the problem quick enought to prevent harm. in these car the chance of something to go wrong is likely. an acciden

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1283: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(



->Answer:
 <pad> Understand the dangers of driverless car. Understand the dangers of driverless car.</s>


In [ ]:
score_command = 'Try score this text on the scale of 1 to 6: {}'.format(input_text)
print('Command:', score_command)

input_ids = tokenizer(score_command, return_tensors = "pt").input_ids.to("cuda")

outputs = model.generate(input_ids)
print('Answer:\n', tokenizer.decode(outputs[0]))

Command: Try score this text on the scale of 1 to 6: driverless car are something that will very soon be a common part of our everyday furture life. these car have proven to be a time saving and fuel saving option. they also allow driver to not have to be in full control a they would have had in normal car today. although these would be some very helpful thing and would be more economically friendly, driverless car prove to be very risking and there are just too many question about ensuring safety for them to be allowed today. driverless car could possibly be dangerous to the driver, other driver, or people walking around on the street. a of now no driverless car have been developed that are completely driverless. at any point the car could run into a problem where it need a person is assistence. this person may not always pay attention like they should and they may not be able to interfere with the problem quick enought to prevent harm. in these car the chance of something to go wrong

In [ ]:
translate_command = 'Translate this text into French: {}'.format(input_text)
print('Command:', translate_command)

input_ids = tokenizer(translate_command, return_tensors = "pt").input_ids.to("cuda")

outputs = model.generate(input_ids)
print('Answer:\n', tokenizer.decode(outputs[0]))

Command: Translate this text into French: driverless car are something that will very soon be a common part of our everyday furture life. these car have proven to be a time saving and fuel saving option. they also allow driver to not have to be in full control a they would have had in normal car today. although these would be some very helpful thing and would be more economically friendly, driverless car prove to be very risking and there are just too many question about ensuring safety for them to be allowed today. driverless car could possibly be dangerous to the driver, other driver, or people walking around on the street. a of now no driverless car have been developed that are completely driverless. at any point the car could run into a problem where it need a person is assistence. this person may not always pay attention like they should and they may not be able to interfere with the problem quick enought to prevent harm. in these car the chance of something to go wrong is likely.